<a href="https://colab.research.google.com/github/JoaoAnjos-DataScience/JoaoAnjos-DataScience/blob/main/Prevendo_os_valores_das_vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Carregar os dados
data = pd.read_csv("/content/Vendas_BabaEletronica.csv", delimiter=',', quotechar='"', header=0)

# Remover colunas específicas
colunas_para_remover = ['Z', 'X']  # Remova as colunas que não quiser, ou seja que não for preciso para o modelo.
data = data.drop(colunas_para_remover, axis=1)

# Reordenar as colunas, estamos reordenando o modelo para que a coluna Alvo que no caso é a venda seja última, para melhorar o modelo.
nova_ordem = ['datavenda', 'mes', 'weekday', 'margem', 'desconto', 'outdesc', 'outmg','venda']
data = data.reindex(columns=nova_ordem)

# Converter a margem para percentual e ajustar a representação
data['margem'] = (data['margem'] * 100).astype(int).map(lambda x: f"{x}%")

# Formatar a coluna "venda" para exibir os valores em Reais (R$) e deixar em branco os valores NaN que no caso são as 31 últimas linhas.
data['venda'] = data['venda'].apply(lambda x: f"R$ {x:.2f}" if not pd.isnull(x) else '')
data


,datavenda,mes,weekday,margem,desconto,outdesc,outmg,venda
0,20220801,agosto,segunda-feira,40%,9190.907534,0,0,R$ 110042.46
1,20220802,agosto,terca-feira,41%,5713.043012,0,0,R$ 58377.32
2,20220803,agosto,quarta-feira,43%,8621.708915,0,0,R$ 64635.39
3,20220804,agosto,quinta-feira,40%,18312.965640,0,0,R$ 140417.32
4,20220805,agosto,sexta-feira,44%,19942.741300,0,0,R$ 149700.29
...,...,...,...,...,...,...,...,...
391,20230827,agosto,domingo,45%,5846.133551,0,0,
392,20230828,agosto,segunda-feira,47%,2657.847261,0,0,
393,20230829,agosto,terca-feira,38%,269.530000,0,0,
394,20230830,agosto,quarta-feira,49%,3434.838118,0,0,


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Carregar os dados
data = pd.read_csv("/content/Vendas_BabaEletronica.csv", delimiter=',', quotechar='"', header=0)

# Substituir NaN por 0 nas últimas 31 linhas da coluna 'vendas'
data.loc[data.index[-31:], 'venda'] = data.loc[data.index[-31:], 'venda'].fillna(0)

# Converter colunas categóricas em formato apropriado
encoder = LabelEncoder()
data['mes'] = encoder.fit_transform(data['mes'])
data['weekday'] = encoder.fit_transform(data['weekday'])


# Separar as features e o alvo
X = data.drop(columns=['datavenda', 'margem', 'outdesc', 'outmg','venda'])  # Remover a data da venda (não é uma feature) e a variável alvo
y = data['venda']

# Dividir os dados em conjunto de treinamento e teste (70% treinamento, 30% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalizar os dados numéricos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo 1: Regressão Linear
linear_reg = LinearRegression()
linear_reg.fit(X_train_scaled, y_train)
linear_reg_predictions = linear_reg.predict(X_test_scaled)
linear_reg_mse = mean_squared_error(y_test, linear_reg_predictions)
linear_reg_rmse = np.sqrt(linear_reg_mse)
linear_reg_mae = mean_absolute_error(y_test, linear_reg_predictions)
linear_reg_r2 = r2_score(y_test, linear_reg_predictions)

# Modelo 2: Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train_scaled, y_train)
rf_reg_predictions = rf_reg.predict(X_test_scaled)
rf_reg_mse = mean_squared_error(y_test, rf_reg_predictions)
rf_reg_rmse = np.sqrt(rf_reg_mse)
rf_reg_mae = mean_absolute_error(y_test, rf_reg_predictions)
rf_reg_r2 = r2_score(y_test, rf_reg_predictions)

# Modelo 3: Gradient Boosting Regressor
gb_reg = GradientBoostingRegressor(random_state=42)
gb_reg.fit(X_train_scaled, y_train)
gb_reg_predictions = gb_reg.predict(X_test_scaled)
gb_reg_mse = mean_squared_error(y_test, gb_reg_predictions)
gb_reg_rmse = np.sqrt(gb_reg_mse)
gb_reg_mae = mean_absolute_error(y_test, gb_reg_predictions)
gb_reg_r2 = r2_score(y_test, gb_reg_predictions)

print("Modelo de Regressão Linear:")
print("  - MSE: {:.2f}".format(linear_reg_mse))
print("  - RMSE: {:.2f}".format(linear_reg_rmse))
print("  - MAE: {:.2f}".format(linear_reg_mae))
print("  - R²: {:.2f}".format(linear_reg_r2))

print("\nModelo de Random Forest Regressor:")
print("  - MSE: {:.2f}".format(rf_reg_mse))
print("  - RMSE: {:.2f}".format(rf_reg_rmse))
print("  - MAE: {:.2f}".format(rf_reg_mae))
print("  - R²: {:.2f}".format(rf_reg_r2))

print("\nModelo de Gradient Boosting Regressor:")
print("  - MSE: {:.2f}".format(gb_reg_mse))
print("  - RMSE: {:.2f}".format(gb_reg_rmse))
print("  - MAE: {:.2f}".format(gb_reg_mae))
print("  - R²: {:.2f}".format(gb_reg_r2))

# Usar o mesmo objeto StandardScaler ajustado com os dados de treinamento
X_teste_scaled = scaler.transform(data.drop(columns=['datavenda', 'margem', 'outdesc', 'outmg','venda']))

# Prever as vendas das últimas 31 linhas usando o modelo de Regressão Linear
linear_reg_predictions_test = linear_reg.predict(X_teste_scaled)
mae_linear_reg_test = mean_absolute_error(data['venda'], linear_reg_predictions_test)
mse_linear_reg_test = mean_squared_error(data['venda'], linear_reg_predictions_test)
rmse_linear_reg_test = np.sqrt(mse_linear_reg_test)
r2_linear_reg_test = r2_score(data['venda'], linear_reg_predictions_test)

print("\nDesempenho do modelo de Regressão Linear nas últimas 31 linhas:")
print("  - MAE: {:.2f}".format(mae_linear_reg_test))
print("  - MSE: {:.2f}".format(mse_linear_reg_test))
print("  - RMSE: {:.2f}".format(rmse_linear_reg_test))
print("  - R²: {:.2f}".format(r2_linear_reg_test))

# Prever as vendas das últimas 31 linhas usando o modelo de Random Forest Regressor
rf_reg_predictions_test = rf_reg.predict(X_teste_scaled)
mae_rf_reg_test = mean_absolute_error(data['venda'], rf_reg_predictions_test)
mse_rf_reg_test = mean_squared_error(data['venda'], rf_reg_predictions_test)
rmse_rf_reg_test = np.sqrt(mse_rf_reg_test)
r2_rf_reg_test = r2_score(data['venda'], rf_reg_predictions_test)

print("\nDesempenho do modelo de Random Forest Regressor nas últimas 31 linhas:")
print("  - MAE: {:.2f}".format(mae_rf_reg_test))
print("  - MSE: {:.2f}".format(mse_rf_reg_test))
print("  - RMSE: {:.2f}".format(rmse_rf_reg_test))
print("  - R²: {:.2f}".format(r2_rf_reg_test))

# Prever as vendas das últimas 31 linhas usando o modelo de Gradient Boosting Regressor
gb_reg_predictions_test = gb_reg.predict(X_teste_scaled)
mae_gb_reg_test = mean_absolute_error(data['venda'], gb_reg_predictions_test)
mse_gb_reg_test = mean_squared_error(data['venda'], gb_reg_predictions_test)
rmse_gb_reg_test = np.sqrt(mse_gb_reg_test)
r2_gb_reg_test = r2_score(data['venda'], gb_reg_predictions_test)

print("\nDesempenho do modelo de Gradient Boosting Regressor nas últimas 31 linhas:")
print("  - MAE: {:.2f}".format(mae_gb_reg_test))
print("  - MSE: {:.2f}".format(mse_gb_reg_test))
print("  - RMSE: {:.2f}".format(rmse_gb_reg_test))
print("  - R²: {:.2f}".format(r2_gb_reg_test))


Modelo de Regressão Linear:
  - MSE: 1213775462.76
  - RMSE: 34839.28
  - MAE: 23862.45
  - R²: 0.29

Modelo de Random Forest Regressor:
  - MSE: 405410492.82
  - RMSE: 20134.81
  - MAE: 14585.79
  - R²: 0.76

Modelo de Gradient Boosting Regressor:
  - MSE: 282608567.48
  - RMSE: 16810.97
  - MAE: 11966.33
  - R²: 0.84

Desempenho do modelo de Regressão Linear nas últimas 31 linhas:
  - MAE: 21745.63
  - MSE: 996645509.40
  - RMSE: 31569.69
  - R²: 0.54

Desempenho do modelo de Random Forest Regressor nas últimas 31 linhas:
  - MAE: 8415.61
  - MSE: 211930776.98
  - RMSE: 14557.84
  - R²: 0.90

Desempenho do modelo de Gradient Boosting Regressor nas últimas 31 linhas:
  - MAE: 8026.74
  - MSE: 138069338.47
  - RMSE: 11750.29
  - R²: 0.94


In [ ]:
# Importar as bibliotecas necessárias
import pandas as pd

# Criar um DataFrame com as previsões
previsoes_df = pd.DataFrame({
    'Data': ultimas_31_linhas['datavenda'],  # Data da venda
    'Previsão Regressão Linear': linear_reg_predictions_ultimas_31_linhas,
    'Previsão Random Forest': rf_reg_predictions_ultimas_31_linhas,
    'Previsão Gradient Boosting': gb_reg_predictions_ultimas_31_linhas,
})

# Converter a margem para percentual e ajustar a representação
previsoes_df['Previsão Regressão Linear'] = previsoes_df['Previsão Regressão Linear'].apply(lambda x: f"R$ {x:.2f}" if not pd.isnull(x) else '')
previsoes_df['Previsão Random Forest'] = previsoes_df['Previsão Random Forest'].apply(lambda x: f"R$ {x:.2f}" if not pd.isnull(x) else '')
previsoes_df['Previsão Gradient Boosting'] = previsoes_df['Previsão Gradient Boosting'].apply(lambda x: f"R$ {x:.2f}" if not pd.isnull(x) else '')

# Mostrar o DataFrame
previsoes_df


,Data,Previsão Regressão Linear,Previsão Random Forest,Previsão Gradient Boosting
365,20230801,R$ 30950.41,R$ 44498.47,R$ 35858.29
366,20230802,R$ 34310.88,R$ 21732.03,R$ -3348.67
367,20230803,R$ 44373.00,R$ 20816.92,R$ 4521.41
368,20230804,R$ 48329.69,R$ 2456.60,R$ 4307.41
369,20230805,R$ 66587.51,R$ 402.28,R$ 11810.15
370,20230806,R$ 70848.49,R$ 0.00,R$ 3499.13
371,20230807,R$ 42382.34,R$ 0.00,R$ 136.17
372,20230808,R$ 48390.85,R$ 0.00,R$ 1179.93
373,20230809,R$ 31259.57,R$ 0.00,R$ -1107.73
374,20230810,R$ 47012.25,R$ 0.00,R$ 1494.01
